# MODIS Aqua ocean products <img align="right" src="../../resources/easi_logo.jpg">

#### Index
- [Overview](#Overview)
- [Setup (dask, imports, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Quality layer (mask)](#Quality-layer)
- [Scaling and nodata](#Scaling-and-nodata)
- [Visualisation](#Visualisation)
- [Appendix](#Appendix)

## Overview

MODIS L2 ocean products available from NASA https://oceandata.sci.gsfc.nasa.gov. These data are processed by NASA's Ocean Biology Processing Group (OBPG).

#### Data source and documentation

Algorithm Theoretical Basis Documents: https://oceancolor.gsfc.nasa.gov/atbd/

| Name | Product | Information |
|------|---------|-------------|
| MODIS Aqua L2 Ocean colour (OC) | nasa_aqua_l2_oc | https://oceancolor.gsfc.nasa.gov/docs/format/l2oc_modis
| MODIS Aqua L2 Inherent optical properties (IOP) | nasa_aqua_l2_iop | https://oceancolor.gsfc.nasa.gov/docs/format/l2iop_modis
| MODIS Aqua L2 See surface temperature (SST) | nasa_aqua_l2_sst | https://oceancolor.gsfc.nasa.gov/docs/format/l2sst_modis

#### EASI pipeline

| Task | Summary |
|------|---------|
| Status | Pre-operational |
| Source | Manual search for scenes in AOI, see https://oceancolor.gsfc.nasa.gov/cgi/browse.pl
| Preprocess | GDAL warp to WGS84, 0.01 x 0.01 deg
| Format | Convert to COGs
| Prepare | EO3 metadata taken from Aqua L2 netcdf file
| TODO |

## Setup

#### Imports

These are a standard set of imports that we use across many notebooks

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
hv.extension("bokeh", logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('../../scripts'))
import notebook_utils

# Hide ShapelyDeprecationWarning
import warnings
warnings.filterwarnings('ignore', message='.+Shapely 2.0')

#### Dask

In [ ]:
cluster, client = notebook_utils.initialize_dask(workers=(1,2), use_gateway=False, wait=True)
display(cluster if cluster else client)
print(notebook_utils.localcluster_dashboard(client))

_ = configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

#### ODC database

In [ ]:
# Template code for development database
# CONF = """
# [datacube]
# db_hostname:
# db_database:
# db_username:
# db_password:
# """
# from datacube.config import read_config, LocalConfig
# dc = datacube.Datacube(config=LocalConfig(read_config(CONF)), env='datacube')

dc = datacube.Datacube()  # Comment if using a development database

#### Example query

Change any of the parameters in the query object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions  + /product (when available)

In [ ]:
# NSW
min_longitude, max_longitude = (150, 160)
min_latitude, max_latitude = (-35, -29)
native_crs = 'epsg:4326'
min_date = '2021-11-15'
max_date = '2021-11-15'
product = 'nasa_aqua_l2_sst'

query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'latitude': 2048, 'longitude': 2048},  # Dask chunks
}

In [ ]:
# Load data
data = dc.load(**query)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')
display(valid_mask)

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query['product']]
notebook_utils.heading(f'Measurement table for product: {query["product"]}')
notebook_utils.display_table(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
notebook_utils.display_table(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    notebook_utils.display_table(masking.describe_variable_flags(data[flag]))

## Quality layer

In [ ]:
# Make L2_FLAGS image
flag_name = 'l2_flags'
if 'sst' in query['product']:
    flag_name = 'qual_sst'
flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)

In [ ]:
# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': cc.rainbow,
    'colorbar': True,
    'width': 1000,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = flag_data.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset CRS
    projection = ccrs.PlateCarree(),         # Output Projection (ccrs.PlateCarree() when coastline=True)
    coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist()

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:
# Create mask layer

if 'oc' in query['product'] or 'iop' in query['product']:
    # https://oceancolor.gsfc.nasa.gov/atbd/ocl2flags/
    # "L3 Mask Default"
    flag_name = 'l2_flags'
    bad_pixel_flags = (
        'ATMFAIL',
        'LAND',
        'HIGLINT',
        'HILT',
        'HISATZEN',
        'STRAYLIGHT',
        'CLDICE',
        'COCCOLITH',
        'HISOLZEN',
        'LOWLW',
        'CHLFAIL',
        'NAVWARN',
        'MAXAERITER',
        'CHLWARN',
        'ATMWARN',
        'NAVFAIL',
    )
    
if 'sst' in query['product']:
    # https://oceancolor.gsfc.nasa.gov/atbd/sst/flag/
    flag_name = 'qual_sst'
    bad_pixel_flags = (
        'Bad',            # 3
        'Not processed',  # 4
    )
    
    # Flags SST
    # Corresponds to qual_sst = 3 Bad or 4 Not processed
    # 'flags_sst' has negative flag values. Its not clear how to handle these.
    # Negative flag values appear to correspond to qual_sst = 3 Bad.
    # flag_name = 'flags_sst'
    # bad_pixel_flags = (
    #    'ISMASKED',     # 4
    #    'BTBAD',        # 4
    #    'VHISENZ',      # 3
    #    'BTRANGE',      # 3
    #    'SSTRANGE',     # 3
    #    'BTVNONUNIF',   # 3
    #    'SSTREFVDIFF',  # 3
    #    'CLOUD',        # 3
    #)

good_pixel_mask = ~enum_to_bool(data[flag_name], bad_pixel_flags)  # -> bool array
display(good_pixel_mask)  # Type: bool

## Scaling and nodata

- https://oceancolor.gsfc.nasa.gov/docs/format/l2oc_modis/
- https://oceancolor.gsfc.nasa.gov/docs/format/l2iop_modis/
- https://oceancolor.gsfc.nasa.gov/docs/format/l2sst_modis/

In [ ]:
# Define a scaling lookup dict

scaling = {
    # modis_aqua_l2_oc
    'aot_869': {'scale': 0.0001, 'offset': 0},             # Aerosol optical thickness at 869 nm
    'angstrom': {'scale': 0.0001, 'offset': 2.5},          # Aerosol Angstrom exponent, 443 to 865 nm\
    'rrs_412': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 412 nm
    'rrs_443': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 443 nm
    'rrs_469': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 469 nm
    'rrs_488': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 488 nm
    'rrs_531': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 531 nm
    'rrs_547': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 547 nm
    'rrs_555': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 555 nm
    'rrs_645': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 645 nm
    'rrs_667': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 667 nm
    'rrs_678': {'scale': 2e-06, 'offset': 0.05},           # Remote sensing reflectance at 678 nm
    'chlor_a': {'scale': 1, 'offset': 0},                  # Chlorophyll Concentration, OCI Algorithm
    'chl_ocx': {'scale': 1, 'offset': 0},                  # Chlorophyll Concentration, OC3 Algorithm
    'kd_490': {'scale': 0.0002, 'offset': 0},              # Diffuse attenuation coefficient at 490 nm, KD2 algorithm
    'pic': {'scale': 2e-06, 'offset': 0.065},              # Calcite Concentration, Balch and Gordon
    'poc': {'scale': 0.2, 'offset': 6400},                 # Particulate Organic Carbon, D. Stramski, 2007 (443/555 version)
    'ipar': {'scale': 1e-07, 'offset': 0},                 # Instantaneous Photosynthetically Available Radiation
    'nflh': {'scale': 8.5e-05, 'offset': 2.25},            # Normalized Fluorescence Line Height
    'par': {'scale': 0.002, 'offset': 65.5},               # Photosynthetically Available Radiation, R. Frouin
    # modis_aqua_l2_iop
    'a_412_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 412 nm, GIOP model
    'a_443_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 443 nm, GIOP model
    'a_469_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 469 nm, GIOP model
    'a_488_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 488 nm, GIOP model
    'a_531_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 531 nm, GIOP model
    'a_547_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 547 nm, GIOP model
    'a_555_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 555 nm, GIOP model
    'a_645_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 645 nm, GIOP model
    'a_667_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 667 nm, GIOP model
    'a_678_giop': {'scale': 0.0001, 'offset': 2.5},        # Total absorption at 678 nm, GIOP model
    'bb_412_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 412 nm, GIOP model
    'bb_443_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 443 nm, GIOP model
    'bb_469_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 469 nm, GIOP model
    'bb_488_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 488 nm, GIOP model
    'bb_531_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 531 nm, GIOP model
    'bb_547_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 547 nm, GIOP model
    'bb_555_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 555 nm, GIOP model
    'bb_645_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 645 nm, GIOP model
    'bb_667_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 667 nm, GIOP model
    'bb_678_giop': {'scale': 5e-06, 'offset': 0.14},       # Total backscattering at 678 nm, GIOP model
    'aph_443_giop': {'scale': 0.0001, 'offset': 2.5},      # Absorption due to phytoplankton at 443 nm, GIOP model
    'adg_443_giop': {'scale': 0.0001, 'offset': 2.5},      # Absorption due to gelbstoff and detrital material at 443 nm, GIOP model
    'adg_s_giop': {'scale': 1, 'offset': 0},               # Detrital and gelbstoff absorption spectral parameter for GIOP model
    'bbp_443_giop': {'scale': 5e-06, 'offset': 0.14},      # Particulate backscattering at 443 nm, GIOP model
    'bbp_s_giop': {'scale': 1, 'offset': 0},               # Backscattering spectral parameter for GIOP model
    'rrsdiff_giop': {'scale': 1, 'offset': 0},             # Fractional mean Rrs difference, GIOP model
    'aph_unc_443_giop': {'scale': 0.0001, 'offset': 2.5},  # Uncertainty in absorption due to phytoplankton at 443 nm, GIOP model
    'adg_unc_443_giop': {'scale': 0.0001, 'offset': 2.5},  # Uncertainty in absorption due to gelbstoff and detrital material at 443 nm, GIOP model
    'bbp_unc_443_giop': {'scale': 5e-06, 'offset': 0.14},   # Uncertainty in particulate backscatter at 443 nm, GIOP model
    # modis_aqua_l2_sst
    'sst': {'scale': 0.005, 'offset': 0},                  # Sea Surface Temperature
    'bias_sst': {'scale': 0.005, 'offset': 0},             # Sea Surface Temperature Bias
    'stdv_sst': {'scale': 0.005, 'offset': 0},             # Sea Surface Temperature Standard Deviation
    'sstref': {'scale': 0.005, 'offset': 0},               # Sea Surface Temperature Reference
}

In [ ]:
# Select a layer and apply masking and scaling, then persist in dask

layer_name = 'rrs_555'  # OC
# layer_name = 'sst'      # SST

# Apply valid mask and good pixel mask
layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scaling[layer_name]['scale'] + scaling[layer_name]['offset']
layer = layer.persist()

## Visualisation

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 1000,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 0.01),                         # rrs_555
    # 'clim': (20, 35),                          # sst
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used
    
layer_plot = layer.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

# Good image of Sarawak: 2020-01-27 0555

## Appendix